In [48]:
import math
import time
import numpy as np
import numpy as np
import sys
from matplotlib import collections as mc
import matplotlib.pyplot as plt
from collections import defaultdict

In [49]:

class Point(object):
    
    def __init__(self, x, y):
        self.x = x
        self.y = y
        
    def str_point(self):
        return "["+str(round(self.x,3))+","+str(round(self.y,3))+"]"
        
    def is_x_greater(self, p):
        return self.x > p.x
    
    def is_y_greater(self, p):
        return self.y > p.y
        
    def delta_x(self, p):
        return self.x-p.x
    
    def delta_y(self, p):
        return self.y-p.y
    
    def dist_to_point(self, p):
        return math.sqrt(math.pow(self.x - p.x, 2) + math.pow(self.y - p.y, 2))
    
    def dist_to_par(self, focus): #se la distanza punto parabola è infinita (direttrice parabola passante per punto e fuoco)
        return math.pow(self.dist_to_point(focus), 2)/(2*abs(self.delta_y(focus))) if self.delta_y(focus)!=0 else math.inf
    
    def is_same_point(self, p):
        return self.x == p.x and self.y == p.y
        
    def is_in(self, ps):
        for p in ps:
            if self.is_same_point(p):
                return True
        return False


In [50]:
def circumcenter(a, b, c):
    if isinstance(a, Segment):
        print("a is", a.str_segment())
    if isinstance(b, Segment):
        print("b is", b.str_segment())
    if isinstance(c, Segment):
        print("c is", c.str_segment())
    d = (a.x - c.x) * (b.y - c.y) - (b.x - c.x) * (a.y - c.y)
    if d==0: #tre punti allineati
        return Point(math.inf, math.inf)
    xc = (((a.x - c.x) * (a.x + c.x) + (a.y - c.y) * (a.y + c.y)) / 2 * 
          (b.y - c.y) - ((b.x - c.x) * (b.x + c.x) + (b.y - c.y) * (b.y + c.y)) / 2 * 
          (a.y - c.y))/d
    yc = (((b.x - c.x) * (b.x + c.x) + (b.y - c.y) * (b.y + c.y)) / 2 * 
          (a.x - c.x) - ((a.x - c.x) * (a.x + c.x) + (a.y - c.y) * (a.y + c.y)) / 2 * 
          (b.x - c.x))/d
    return Point(xc, yc)

def circumbottom(a, b, c):
    cc = circumcenter(a, b, c)
    return Point(cc.x, cc.y-cc.dist_to_point(a)) #(cc.x, cc.y-r)

def circumtop(a, b, c):
    cc = circumcenter(a, b, c)
    return Point(cc.x, cc.y+cc.dist_to_point(a)) #(cc.x, cc.y-r)

def arc_cross_x(f1, f2, q):
    if f1.y!=f2.y:
        s1=(f1.y*f2.x-f1.x*f2.y+f1.x*q-f2.x*q+math.sqrt((f1.x*f1.x+f1.y*f1.y-2*f1.x*f2.x+f2.x*f2.x-2*f1.y*f2.y+f2.y*f2.y)*(f1.y*f2.y-f1.y*q-f2.y*q+q*q)))/(f1.y-f2.y)  
        s2=(f1.y*f2.x-f1.x*f2.y+f1.x*q-f2.x*q-math.sqrt((f1.x*f1.x+f1.y*f1.y-2*f1.x*f2.x+f2.x*f2.x-2*f1.y*f2.y+f2.y*f2.y)*(f1.y*f2.y-f1.y*q-f2.y*q+q*q)))/(f1.y-f2.y)  
        return [s1, s2] if s1<s2 else [s2, s1]
    else:
        return [(f1.x+f2.x)/2, (f1.x+f2.x)/2]

def parabola_y(f, q, x):
    if q!=f.y:
        return math.pow((x - f.x),2)/(2*(abs(q-f.y)))+(q-f.y)/2
    else:
        return math.inf

def distance(a, b):
    return math.sqrt(math.pow((a.x - b.x), 2) + math.pow((a.y - b.y), 2))

def go_to_i(alist, test):
    i = 0
    while i < len(alist) and test(alist[i]):
        i += 1
    return i

In [51]:

class Segment(object):
    
    def __init__(self,  pl=None, pr=None, start=None, end=None):
        self.start = start #punto iniziale del segmento
        self.end = end #punto finale del segmento
        self.pl = pl #uno dei due punti di cui l'oggetto è un segmento dell'asse
        self.pr = pr #uno dei due punti di cui l'oggetto è un segmento dell'asse
        
    def str_segment(self):
        start="Start: None" if self.start==None else "Start: "+self.start.str_point()
        end="Theta: "+str(self.theta()) if self.end==None else "End: "+self.end.str_point()
        return start+"; "+end+"; left arc focus: "+self.pl.str_point()+"; right arc focus: "+self.pr.str_point()
    
    def halfplane(self):
        #QUI SFRUTTA L'ORDINAMENTO IN Y DEGLI EVENTI: i semisegmenti vengono specchiati rispetto a x se cambia il verso di sweep
        #per sweep verso x positive. +: semipiano superiore e x+; -: semipiano inferiore e x-
        #per sweep verso x negative. +: semipiano inferiore e x+; -: semipiano superiore e x- (possibile invertire la condizione)
        #L'assegnazione di x+ e x- è stata scelta arbitrariamente
        if self.pl.x<self.pr.x: #semipiano superiore
            return 2
        if self.pl.x>self.pr.x: #semipiano inferiore
            return -2
        if self.pl.x==self.pr.x and self.pl.y>self.pr.y: #asse x+
            return 1
        return -1 #asse x-
    
    def m(self): #y=mx+q ; m=-(pl.x-pr.x)/(pl.y-pr.y) ; q=(pl.y+pr.y)/2-m*(pl.x+pr.x)/2
        return math.inf if self.pl.y==self.pr.y else -(self.pl.x-self.pr.x)/(self.pl.y-self.pr.y)
    
    def direction(self):
        (m, hp)=(self.m(), self.halfplane())
        return Point(1,m) if (hp*m>0 or (m==0 and hp>0)) else Point(-1,-m) 
    
    def theta(self, rad=False):
        theta = math.atan2(self.direction().y, self.direction().x)
        return math.degrees(theta) if rad==False else theta
        
    def y(self, x):
        return self.m()*x+self.start.y-self.m()*self.start.x
    
    def get_intersection(self, s1, s2=None):
        s2=self if s2==None else s2
        (s1_m, s2_m)=(s1.m(), s2.m())
        (s1_hp, s2_hp)=(s1.halfplane(), s2.halfplane())
        #print(str(s1_m), str(s2_m), str(s1_hp), str(s2_hp))
        if (s1_hp*s1_m>0 or (s1_m==0 and s1_hp>0)): #semiretta semipiano dx 
            s1_vec=Point(1,s1_m) 
        else: #semiretta nel semipiano sx
            s1_vec=Point(-1,-s1_m)
        if (s2_hp*s2_m>0 or (s2_m==0 and s2_hp>0)): #semiretta semipiano dx 
            s2_vec=Point(1,s2_m) 
        else: #semiretta nel semipiano sx
            s2_vec=Point(-1,-s2_m)
        dx = s2.start.x - s1.start.x
        dy = s2.start.y - s1.start.y
        if s2_m==math.inf: #segmento 2 verticale
            #print("Vertical:", s2_hp*(s1_m*dx-dy), s1_vec.x*dx)
            if s2_hp*(s1_m*dx-dy)>=-1.0e-10 and s1_vec.x*dx>=-1.0e-10:
                return Point(s2.start.x, s1.start.y+s1_m*(dx))
            return None
        if s1_m==math.inf: #segmento 1 verticale; stesse condizioni ma con s1->s2, dx->-dx, dy->-dy
            #print("Vertical ",s1_hp*(s2_m*dx-dy), s2_vec.x*dx)
            if s1_hp*(s2_m*dx-dy)<=1.0e-10 and s2_vec.x*dx<=1.0e-10:
                return Point(s1.start.x, s2.start.y-s2_m*(dx))
            return None
        det = s2_vec.x * s1_vec.y - s2_vec.y * s1_vec.x
        if det==0:
            return None
        u = (dy * s2_vec.x - dx * s2_vec.y) / det
        v = (dy * s1_vec.x - dx * s1_vec.y) / det
        #print("Not vertical ", str(u), str(v))
        if (u>=-1e-10 and v>=1e-10) or (u>=1e-10 and v>=-1e-10):
            return Point(s1.start.x+u*s1_vec.x, s1.start.y+u*s1_vec.y)

In [52]:
class Beachline(object):
    
    def __init__(self, item=None):
        self.item = item
        self.parent = None
        self.l = None
        self.r = None
        self.c_event=None
        
    def str_node(self, directions=[]):
        cur_node=self
        for d in directions:
            cur_node=cur_node.l if d=="l" else cur_node.r
        if isinstance(cur_node.item, Segment):
            return "Edge. "+cur_node.item.str_segment()
        else:
            return "Arc. "+cur_node.item.str_point()+". Circle event: None" if cur_node.c_event==None else "Arc. "+cur_node.item.str_point()+". Event: "+cur_node.c_event.str_event()
    
    def str_tree(self, path="Path: root"):
        if self.item==None: #un po' inefficente fargli compiere l'operazione ad ogni iterazione
            return "Empty beachline.\n"
        node_string=path+". "+self.str_node()+"\n"
        if not self.is_leaf():
            node_string+=self.l.str_tree(path+"->l")
            node_string+=self.r.str_tree(path+"->r")
        return node_string
    
    def get_leaves(self):
        cur_node=self
        leaves=[]
        while not cur_node.is_leaf():
            cur_node=cur_node.l
        while cur_node!=None:
            leaves.append(cur_node)
            cur_node=cur_node.get_rleaf_and_rparent()[0]
        return leaves
              
    def is_leaf(self):
        return self.l==None
    
    def is_lchild(self):
        return not self.is_root() and self.parent.l==self
    
    def is_rchild(self):
        return not self.is_root() and self.parent.r==self
                
    def is_root(self):
        return self.parent==None
        
    def add_rchild(self, item, event=None):
        child = Beachline(item)
        child.parent = self
        self.r = child
        
    def add_lchild(self, item, event=None):
        child = Beachline(item)
        child.parent = self
        self.l = child
        
    def get_nearest_arc_node_by_x(self, new_site):
        if self.item==None: #beachline vuota. In tal caso non esiste un sito più vicino
            return None
        else:
            cur_node=self
        while isinstance(cur_node.item, Segment): #essendo al primo passaggio, siteA.x!=siteB.x, per ogni sito
            cur_node=cur_node.l if new_site.x<(cur_node.item.pl.x+cur_node.item.pr.x)/2 else cur_node.r
        return cur_node
    
    def get_arc_node_on_site(self, new_site):
        if self.item==None: #beachline vuota. In tal caso non esiste un arco sopra il sito aggiunto, Non dovrebbe verificarsi mai
            return None
        else:
            cur_node=self
        cur_node = self
        while isinstance(cur_node.item, Segment): #se il nuovo sito è equidistante da pl e pr è automaticamente associato al figlio destro (non so se corretto)
            #print("Intersection between",cur_node.item.pl.str_point(),"and",cur_node.item.pr.str_point(),"when q=",str(new_site.y),": ", str(arc_cross_x(cur_node.item.pl, cur_node.item.pr, new_site.y)))
            sol=0 if cur_node.item.pl.y<cur_node.item.pr.y else 1
            cur_node=cur_node.l if new_site.x<(arc_cross_x(cur_node.item.pl, cur_node.item.pr, new_site.y)[sol]) else cur_node.r
        return cur_node
    
    def get_lleaf_and_lparent(self, leaf=None):
        cur_node=self if leaf==None else leaf #se non passo un argomento al metodo utilizza come foglia il nodo chiamante
        while cur_node.is_lchild():
            cur_node=cur_node.parent
        if cur_node.is_root():
            return None, None
        ledge=cur_node.parent
        cur_node=cur_node.parent.l
        while not cur_node.is_leaf():
            cur_node=cur_node.r
        return cur_node, ledge
    
    def get_rleaf_and_rparent(self, leaf=None):
        cur_node=self if leaf==None else leaf #se non passo un argomento al metodo utilizza come foglia il nodo chiamante
        while cur_node.is_rchild():
            cur_node=cur_node.parent
        if cur_node.is_root():
            return None, None
        redge=cur_node.parent
        cur_node=cur_node.parent.r
        while not cur_node.is_leaf():
            cur_node=cur_node.l
        return cur_node, redge  
    
    #Deve essere chiamato per aggiungere gli archi iniziali, cioè quagli archi associati ai primi eventi sito incontrati alla stessa y
    #Ha all'incirca l'effetto cumulato di arr_arc seguito da rm_arc, senza la parte di gestione degli eventi cerchio. Introduce una 
    #lieve asimmetria rispetto alla restante interfaccia in quanto non si fa passare il nodo del segmento modificato (come fa rm_arc),
    #siccome tale dato non serve già al metodo più esterno che gestisce gli eventi sito
    def add_start_arc(self, new_arc_site, nearest_arc_node=None): 
        if nearest_arc_node==None: #mi è stato passato il primo sito in assoluto
            self.item=new_arc_site
            return
        nearest_arc_site=nearest_arc_node.item
        if new_arc_site.x<nearest_arc_site.x: #nuovo sito inserito come figlio sx; sito più vicino in x come figlio dx
            #aggiornamento nodo segmento che tracciava l'asse tra i due siti fra cui è stato inserito il nuovo sito
            top_updated_edge_node=nearest_arc_node.get_lleaf_and_lparent()[1]
            if top_updated_edge_node!=None: #se il sito inserito non è quello più a sx
                top_updated_edge_node.item.pr=new_arc_site
                top_updated_edge_node.item.start.x=(new_arc_site.x+top_updated_edge_node.item.pl.x)/2
            #costruzione del nuovo ramo
            new_edge=Segment(pl=new_arc_site, pr=nearest_arc_site, start=Point((new_arc_site.x+nearest_arc_site.x)/2, -math.inf))
            nearest_arc_node.item=new_edge
            nearest_arc_node.add_lchild(new_arc_site)
            nearest_arc_node.add_rchild(nearest_arc_site)
        else: #nuovo sito inserito come figlio dx; sito più vicino in x come figlio sx
            top_updated_edge_node=nearest_arc_node.get_rleaf_and_rparent()[1]
            if top_updated_edge_node!=None:
                top_updated_edge_node.item.pl=new_arc_site
                top_updated_edge_node.item.start.x=(new_arc_site.x+top_updated_edge_node.item.pr.x)/2
            new_edge=Segment(pl=nearest_arc_site, pr=new_arc_site, start=Point((new_arc_site.x+nearest_arc_site.x)/2, -math.inf))
            nearest_arc_node.item=new_edge
            nearest_arc_node.add_rchild(new_arc_site)
            nearest_arc_node.add_lchild(nearest_arc_site)
            
    #modifica la beach in modo che, dove prima c'era il sito arco intersecato, ora si ha sito arco intersecato,
    #segmento con inizio in intersezione, nuovo sito, segmento con inizio in intersezione, sito arco intersecato
    def add_arc(self, new_arc_site, crossed_arc_node):
        crossed_arc_site = crossed_arc_node.item
        #QUI SFRUTTA L'ORDINAMENTO IN Y DEGLI EVENTI: va aggiunto a new_arc_site.y un vettore di lung=new_arc_site.dist_to_par(crossed_arc_site) nel verso opposto a quello di sweep
        #se la sl va verso y crescenti la y del circle event sarà cc.y-r
        #se la sl va verso y decrescenti la y del circle event sarà cc.y+r
        cross_point = Point(new_arc_site.x, new_arc_site.y-new_arc_site.dist_to_par(crossed_arc_site))
        new_lhalfedge=Segment(pl=crossed_arc_site, pr=new_arc_site, start=cross_point)
        new_rhalfedge=Segment(pl=new_arc_site, pr=crossed_arc_site, start=cross_point)
        crossed_arc_node.item = new_lhalfedge
        crossed_arc_node.add_lchild(crossed_arc_site)
        crossed_arc_node.add_rchild(new_rhalfedge)
        crossed_arc_node.r.add_lchild(new_arc_site)
        crossed_arc_node.r.add_rchild(crossed_arc_site)
     
    def rm_arc(self, new_edge_start, arcs_nodes, edges_nodes):#in realtà potrebbe calcolarseli tutti, ma mi sembrava più efficiente passarglieli siccome già noti
        #salvataggi dei valori da ritornare prima dell'aggiornamento dell'albero
        (live_larc_node, dead_arc_node, live_rarc_node)=(arcs_nodes[0], arcs_nodes[1], arcs_nodes[2])
        #aggiornamento dell'albero
            #salvataggi dei riferimenti per nuovo assegnamento
        if dead_arc_node.is_rchild():
            (live_cut_branch, mid_edge_node, top_edge_node)=(edges_nodes[0].l, edges_nodes[0], edges_nodes[1])  
        else:
            (live_cut_branch, mid_edge_node, top_edge_node)=(edges_nodes[1].r, edges_nodes[1], edges_nodes[0])  
            #nuovo assegnamento
        top_edge_node.item = Segment(pl=live_larc_node.item, pr=live_rarc_node.item, start=new_edge_start, end=None)
        if mid_edge_node.is_rchild(): #aggiornamento puntatore padre->figlio
            mid_edge_node.parent.r=live_cut_branch #non usa metodo add_rchild perchè non aggiunge un nodo a partire da un item
        else:
            mid_edge_node.parent.l=live_cut_branch
        live_cut_branch.parent=mid_edge_node.parent #aggiornamento puntatore figlio->padre

In [53]:

class Event(object):
    
    def __init__(self, arc_site_or_nodes): #sites: tupla dei siti associati agli archi coinvolti nell'evento (ordinati)
        if isinstance(arc_site_or_nodes, Point): 
            self.coord=arc_site_or_nodes
            self.arcs_nodes=None
        else:
            self.arcs_nodes=[arc_site_or_nodes[0], arc_site_or_nodes[2], arc_site_or_nodes[4]]
            self.edges_nodes=[arc_site_or_nodes[1], arc_site_or_nodes[3]]
            self.vertex_coord=circumcenter(self.arcs_nodes[0].item, self.arcs_nodes[1].item, self.arcs_nodes[2].item)
            #QUI SFRUTTA L'ORDINAMENTO IN Y DEGLI EVENTI: se sweep verso y crescenti uso circumtop, viceversa circumbottom
            self.coord=Point(self.vertex_coord.x,self.vertex_coord.y+self.arcs_nodes[0].item.dist_to_point(self.vertex_coord))
            
    def str_event(self):
        if(self.arcs_nodes==None): #site event
            return "Site event. Site: "+self.coord.str_point()  
        else: #circle event
            return "Circle event. Coord:"+self.coord.str_point()+". Arc site:"+self.arcs_nodes[1].item.str_point()
    
    def is_same_event(self, e):
        return self.arcs_nodes==e.arcs_nodes and self.coord==e.coord
    
    def is_coord_y_greater(self, e):
        return self.coord.y > e.coord.y+1e-10
    
    #QUI DEFINISCE L'ORDINAMENTO IN Y DEGLI EVENTI: in base a come ordino gli eventi, la sl andrà verso y crescenti o decrescenti
    #inserisce i nuovi circle come primi eventi a tale y. Penso sia giusto gestirli prima dei site event, in modo da avere la beach aggiornata
    def add(self, es):  
        es.insert(go_to_i(es, self.is_coord_y_greater), self)
    
    def rm(self, es):
        i = go_to_i(es, self.is_coord_y_greater)
        while True:
            #se non esiste un altro evento alla stessa y elimina subito l'evento corrente, se no effettua il controllo
            if i==len(es)-1 or self.coord.y != es[i+1].coord.y or self.is_same_event(es[i]): 
                return es.pop(i)
            i+=1  

In [54]:
class Voronoi(object):
    
    def __init__(self, sites):
        self.sites=sites  #siti di V, obj Point
        self.beach=Beachline()
        self.iterations=0
        self.edges=[]
        self.events=[]
        for site in sites:
            Event(site).add(self.events)
             
    
    #rimuove dalla lista degli eventi l'evento cicle il cui arco associato all'indice della beachline argomento è 
    #quello centrale
    def rm_circle_event(self, arc_node):
        if arc_node.c_event != None:
            arc_node.c_event.rm(self.events)
            arc_node.c_event = None
    
    #NB: un circle event è originato da tre siti di V adiacenti. In esso viene eliminato l'arco centrale. Da notare
    #che l'arco centrale non è associabile a priori in modo banale ad uno dei tre punti
    #Il metodo controlla gli archi interessati da un cambiamento della beachline e verifica se ci sono dei circle
    #event loro associati. Se ci sono, aggiunge l'evento alla coda salvando i siti degli archi nell'ordine di comparsa
    #e salvando un riferimento all'evento 
    def add_circle_event(self, arc_node, sweep_y):
        larc_node, ledge_node = arc_node.get_lleaf_and_lparent()
        rarc_node, redge_node = arc_node.get_rleaf_and_rparent()
        if larc_node==None or rarc_node==None:
            return #"No event: first or last arc"
        if larc_node.item.is_same_point(rarc_node.item):
            return #"No event: lefter arc site equals righter arc site"
        cc = circumcenter(larc_node.item, arc_node.item, rarc_node.item)
        if cc.y==math.inf:
            return #"No event: the three arcs sites are on the same line"
        #QUI SFRUTTA L'ORDINAMENTO IN Y DEGLI EVENTI: va preso l'estremo della circonferenza nella direzione di sweep
        #se la sl va verso y crescenti la y del circle event sarà cc.y+r
        #se la sl va verso y decrescenti la y del circle event sarà cc.y-r
        event_point=Point(cc.x, cc.y+arc_node.item.dist_to_point(cc))
        #QUI SFRUTTA L'ORDINAMENTO IN Y DEGLI EVENTI: un evento è sotto la sl se deve essere ancora incontrato durante lo sweep, e viceversa
        #se la sl va verso y crescenti un evento è sotto la sl se ha y minore di quella della sl
        #se la sl va verso y decrescenti un evento è sotto la sl se ha y maggiore di quella della sl
        if event_point.y<sweep_y-1e-10:
            return #"No event: circle event y", event_point.y ,"is below sweepline y",sweep_y
        if ledge_node.item.get_intersection(redge_node.item)==None:
            return #"Edges do not cross"
        arc_node.c_event = Event([larc_node, ledge_node, arc_node, redge_node, rarc_node])
        arc_node.c_event.add(self.events)
        #return arc_node.c_event
    
    def manage_start_site_event(self):
        event=self.events.pop(0)
        self.beach.add_start_arc(event.coord, self.beach.get_nearest_arc_node_by_x(event.coord))
    
    def manage_site_event(self):
        event=self.events.pop(0) #tolgo subito l'evento gestito; facendolo dopo potrei rimuovere un circle event aggiunto
        crossed_arc_node=self.beach.get_arc_node_on_site(event.coord)
        rarc_node=crossed_arc_node.get_rleaf_and_rparent()[0]
        larc_node=crossed_arc_node.get_lleaf_and_lparent()[0]
        #rimozione evento dal nodo che diventerà radice del ramo aggiunto all'albero binario
        self.rm_circle_event(crossed_arc_node)
        #eliminazione eventi associati a foglia destra e sinistra nella vecchia configurazione. Necessaria perchè gli eventi in
        #esse immagazzinati hanno un riferimento al nodo dell'arco intersecato, che ora diventerà il nodo segmento radice
        if larc_node!=None:
            self.rm_circle_event(larc_node)
        if rarc_node!=None:
            self.rm_circle_event(rarc_node)
        self.beach.add_arc(event.coord, crossed_arc_node)
        #aggiornamento degli eventi associati a foglia destra e sinistra nella vecchia configurazione. In tale modo sono assegnati
        #all'oggetto evento i riferimenti ai nodi arco aggiornati
        if larc_node!=None:
            self.add_circle_event(larc_node, event.coord.y)
        if rarc_node!=None:
            self.add_circle_event(rarc_node, event.coord.y)
        #controllo esistenza ed eventuale aggiunta eventi di cerchio dei due archi in cui è diviso l'arco intersecato
        self.add_circle_event(crossed_arc_node.l, event.coord.y)
        self.add_circle_event(crossed_arc_node.r.r, event.coord.y)
        
    def manage_circle_event(self):
        event=self.events.pop(0) #tolgo subito l'evento gestito; facendolo dopo potrei rimuovere un circle event aggiunto
        (dead_edgeA, dead_edgeB)=(event.edges_nodes[0].item, event.edges_nodes[1].item)
        (dead_edgeA.end, dead_edgeB.end)=(event.vertex_coord, event.vertex_coord) 
        self.beach.rm_arc(event.vertex_coord, event.arcs_nodes, event.edges_nodes)
        self.edges.append(dead_edgeA)
        self.edges.append(dead_edgeB)
        self.rm_circle_event(event.arcs_nodes[0])
        self.rm_circle_event(event.arcs_nodes[2])
        self.add_circle_event(event.arcs_nodes[0], event.coord.y)
        self.add_circle_event(event.arcs_nodes[2], event.coord.y)
        
    #se eseguito al temine di run() estrae le semirette presenti nell'albero ad algorimo terminato. Le semirette (segmenti non t
    #terminati) sono quelli presenti a inizio lista edges con start.y=-np.inf (associate ai siti con stessa y gestiti nei primi step)
    #e quelli presenti nell'albero ad algoritmo terminato 
    def extract_edges(self, root=None):
        root=self.beach if root==None else root
        if not root.is_leaf():
            (xs, ys, t) = (root.item.start.x, root.item.start.y, root.item.theta(rad=True))
            root.item.end = Point(xs+100*np.cos(t), ys+100*np.sin(t))
            self.edges.append(root.item)
            self.extract_edges(root.l)
            self.extract_edges(root.r)
            
    def run(self):
        first_y=v2.events[0].coord.y
        while v2.events!=[] and v2.events[0].coord.y==first_y:
            v2.manage_start_site_event()
        while v2.events!=[]:
            v2.manage_site_event() if v2.events[0].arcs_nodes==None else v2.manage_circle_event()
        self.extract_edges()
            
    def plot(self, xrange=(-2,2), yrange = (-2,2), file_name="voronoi_nb.png"):
        self.run()
        lines = [[(edge.start.x, edge.start.y),(edge.end.x, edge.end.y)] for edge in self.edges]
        lc = mc.LineCollection(lines)
        fig, ax = plt.subplots()
        ax.axis([*xrange, *yrange])
        ax.add_collection(lc)
        ax.margins(0.1)
        xs, ys = zip(*[[p.x,p.y] for p in self.sites])
        ax.plot(xs, ys, 'ro')
        fig.savefig(file_name)

In [55]:
v_two_points=[Point(0,-1), Point(0,1)]
v_four_points=[Point(0,0), Point(0,1), Point(0,2), Point(0,3)]
v_triangle=[Point(0,0), Point(1,1), Point(-1,1)]
v_square=[Point(0,-1), Point(1,0), Point(-1,0), Point(0,1)]
v_centered_square=[Point(0,0), Point(0,-1), Point(1,0), Point(-1,0), Point(0,1)]
v_cross=[Point(0,-1), Point(1,0), Point(-1,0), Point(0,2)]
v_octagon=[Point(0,-2), Point(-math.sqrt(2),-math.sqrt(2)), Point(math.sqrt(2),-math.sqrt(2)), Point(-2,0), Point(2,0),Point(-math.sqrt(2),math.sqrt(2)), Point(math.sqrt(2),math.sqrt(2)), Point(0,2)]
v_centered_octagon=[Point(0,0), Point(0,-2), Point(-math.sqrt(2),-math.sqrt(2)), Point(math.sqrt(2),-math.sqrt(2)), Point(-2,0), Point(2,0),Point(-math.sqrt(2),math.sqrt(2)), Point(math.sqrt(2),math.sqrt(2)), Point(0,2)]

In [56]:
v2=Voronoi(v_octagon)
v2.plot()

In [57]:
#v2.extract_edges()
[print(edge.str_segment()) for edge in v2.edges]
#print(v2.beach.str_tree())

Start: [-1.414,0.586]; End: [0.0,0.0]; left arc focus: [-1.414,1.414]; right arc focus: [-2,0]
Start: [-2,-0.828]; End: [0.0,0.0]; left arc focus: [-2,0]; right arc focus: [-1.414,-1.414]
Start: [0.0,0.0]; End: [-0.0,0.0]; left arc focus: [-1.414,1.414]; right arc focus: [-1.414,-1.414]
Start: [-1.414,-3.414]; End: [-0.0,0.0]; left arc focus: [-1.414,-1.414]; right arc focus: [0,-2]
Start: [-0.0,0.0]; End: [0.0,0.0]; left arc focus: [-1.414,1.414]; right arc focus: [0,-2]
Start: [1.414,-3.414]; End: [0.0,0.0]; left arc focus: [0,-2]; right arc focus: [1.414,-1.414]
Start: [0.0,0.0]; End: [-0.0,-0.0]; left arc focus: [-1.414,1.414]; right arc focus: [1.414,-1.414]
Start: [2,-0.828]; End: [-0.0,-0.0]; left arc focus: [1.414,-1.414]; right arc focus: [2,0]
Start: [-0.0,-0.0]; End: [0.0,0.0]; left arc focus: [-1.414,1.414]; right arc focus: [2,0]
Start: [1.414,0.586]; End: [0.0,0.0]; left arc focus: [2,0]; right arc focus: [1.414,1.414]
Start: [0.0,0.0]; End: [0.0,-0.0]; left arc focus: [-

[None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None]